<a href="https://colab.research.google.com/github/oztuncbilek/urban-heat-island/blob/main/notebooks/01_data_acquisition_geemap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Acquisition

In [ ]:
!pip install geemap google-cloud-storage rasterio matplotlib > /dev/null 2>&1

In [ ]:
import ee
import geemap
import os
from google.colab import drive
import rasterio
import matplotlib.pyplot as plt


drive.mount('/content/drive')
work_dir = '/content/drive/MyDrive/UHI-Detection-Analysis/data/raw/'
os.makedirs(work_dir, exist_ok=True)


In [ ]:
# GEE kimlik doğrulama
ee.Authenticate()
ee.Initialize(project='manifest-pride-258211')

In [ ]:
# Hamburg koordinatları
hamburg = ee.Geometry.Point(9.99, 53.55)

# Landsat 8 koleksiyonunu filtrele (2015 ve 2024 yaz ayları)
def get_landsat_data(year):
    collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterBounds(hamburg) \
        .filterDate(f'{year}-06-01', f'{year}-08-31') \
        .filter(ee.Filter.lt('CLOUD_COVER', 10)) \
        .sort('CLOUD_COVER') \
        .first()
    return collection

# LST hesaplama fonksiyonu (basitleştirilmiş)
def calculate_lst(image):
    lst = image.expression(
        '(TIRS1 * 0.00341802 + 149.0) - 273.15',  # Kelvin'den Celsius'a
        {'TIRS1': image.select('ST_B10')}
    ).rename('LST')
    return lst

# 2015 ve 2025 için LST görüntülerini al
lst_2015 = calculate_lst(get_landsat_data(2015))
lst_2024 = calculate_lst(get_landsat_data(2024))

In [ ]:
# Interactive harita oluştur
Map = geemap.Map(center=[53.55, 9.99], zoom=12)

# LST görüntülerini haritaya ekle (renk paleti: 'inferno' veya 'coolwarm')
vis_params = {
    'min': 20,  # Min LST (°C)
    'max': 40,  # Max LST (°C)
    'palette': ['blue', 'green', 'yellow', 'red']  # Veya 'inferno'
}

Map.addLayer(lst_2015, vis_params, 'LST 2015')
Map.addLayer(lst_2024, vis_params, 'LST 2025')

# Layer kontrol paneli ekle
Map.addLayerControl()
Map

In [ ]:
# GeoTIFF olarak export etme fonksiyonu (GÜNCEL)
def export_to_drive(image, name, folder):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=name,
        folder=folder.replace('/content/drive/MyDrive/', ''),  # GEE için göreli yol
        fileNamePrefix=name,
        scale=30,
        region=hamburg.buffer(5000).bounds(),
        fileFormat='GeoTIFF'
    )
    task.start()
    return task

work_dir = 'raw'

# Drive'a kaydet (work_dir kullanarak)
export_to_drive(lst_2015, 'LST_2015_Hamburg', work_dir)
export_to_drive(lst_2024, 'LST_2024_Hamburg', work_dir)

# Task'lerin tamamlanmasını bekle
import time
while True:
    tasks = ee.batch.Task.list()
    if all(task.status()['state'] in ('COMPLETED', 'FAILED') for task in tasks):
        break
    time.sleep(10)
print(f"Export işlemleri tamamlandı!")

In [ ]:
!pip install nbstripout > /dev/null 2>&1

In [ ]:
!nbstripout /content/drive/MyDrive/GitHub_Repos/urban-heat-island/notebooks/01_data_acquisition_geemap.ipynb